In [ ]:
# Setup the environment
import sys
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())


# Setup tools
import setup_app as app
app.setup_tools()
app.setup_tools('neo')
app.setup_tools('planner')

# Setup the environment directories
data_path = os.getenv('DATA_PATH')
dbfs_path = os.getenv('DBFS_PATH')
# Import packages
import driver_tools as neo
from datetime import datetime, date, time, timedelta

import pandas as pd
import json
import shutil

import get_planner as planner
def str_to_bool(s):
    return s.lower() == 'true'
class OptionsAccessor:
    def __init__(self, options):
        self.options = options

    def get(self, category, *keys):
        # Traverse the options dictionary using the category and keys
        value = self.options.get(category, {})
        for key in keys:
            if isinstance(value, dict):  # Ensure we can continue traversing
                value = value.get(key, {})
            else:  # Stop if we reach a non-dict type before finding all keys
                return None
        return value

class AccessOptionsAccessor:
    def __init__(self, options_accessor):
        self.options_accessor = options_accessor

    def get(self, key):
        return self.options_accessor.get('access_options', key)


class RunOptionsAccessor:
    def __init__(self, options_accessor):
        self.options_accessor = options_accessor

    def get(self, key):
        return self.options_accessor.get('default_run_options', key)


class InitOptionsAccessor:
    def __init__(self, options_accessor):
        self.options_accessor = options_accessor

    def get(self, key):
        return self.options_accessor.get('init_options', key)


class LocalCalendarOptionsAccessor:
    def __init__(self, init_options_accessor):
        self.init_options_accessor = init_options_accessor

    def get(self, *keys):
        return self.init_options_accessor.get('local_calendar_options', *keys)


class LocalPlannerOptionsAccessor:
    def __init__(self, init_options_accessor):
        self.init_options_accessor = init_options_accessor

    def get(self, *keys):
        return self.init_options_accessor.get('local_planner_options', *keys)
    
class LocalCurriculumOptionsAccessor:
    def __init__(self, init_options_accessor):
        self.init_options_accessor = init_options_accessor

    def get(self, *keys):
        return self.init_options_accessor.get('local_curriculum_options', *keys)

def load_options_from_file(filepath):
    with open(filepath, 'r') as file:
        options = json.load(file)
    return options

# Call the accessor classes for options
def get_options_accessor(options):
    options_accessor = OptionsAccessor(options)
    access_options_accessor = AccessOptionsAccessor(options_accessor)
    run_options_accessor = RunOptionsAccessor(options_accessor)
    init_options_accessor = InitOptionsAccessor(options_accessor)
    local_calendar_options_accessor = LocalCalendarOptionsAccessor(init_options_accessor)
    local_planner_options_accessor = LocalPlannerOptionsAccessor(init_options_accessor)
    local_curriculum_options_accessor = LocalCurriculumOptionsAccessor(init_options_accessor)
    return (options_accessor, access_options_accessor, run_options_accessor,
            init_options_accessor, local_calendar_options_accessor, local_planner_options_accessor, local_curriculum_options_accessor)
class LabelsAccessor:
    def __init__(self, labels, type='calendar'):  # Default type is 'calendar'
        self.labels = labels
        self.type = type  # 'calendar', 'planner' or 'curriculum'

    def get(self, category, key):
        # Construct the full key based on the type
        full_key = f'local_{self.type}_{category}'
        return self.labels.get(full_key, {}).get(key, None)

class NodeLabelsAccessor(LabelsAccessor):
    def __init__(self, labels_accessor):
        super().__init__(labels_accessor.labels, labels_accessor.type)  # Pass the type to the parent class

    def get(self, key):
        return super().get('node_labels', key)  # Use 'node_labels' as the category

class NodePropertiesAccessor(LabelsAccessor):
    def __init__(self, labels_accessor):
        super().__init__(labels_accessor.labels, labels_accessor.type)

    def get(self, key):
        return super().get('node_properties', key)

class HierarchyLabelsAccessor(LabelsAccessor):
    def __init__(self, labels_accessor):
        super().__init__(labels_accessor.labels, labels_accessor.type)

    def get(self, key):
        return super().get('hierarchy_labels', key)

class HierarchyPropertiesAccessor(LabelsAccessor):
    def __init__(self, labels_accessor):
        super().__init__(labels_accessor.labels, labels_accessor.type)

    def get(self, key):
        return super().get('hierarchy_properties', key)
# Function to load labels from a JSON file
def load_labels_from_file(filepath):
    with open(filepath, 'r') as file:
        labels = json.load(file)
    return labels

# Call the accessor classes
def get_planner_labels_accessor(labels):
    planner_labels_accessor = LabelsAccessor(labels, type='planner')
    node_labels_accessor = NodeLabelsAccessor(planner_labels_accessor)
    node_properties_accessor = NodePropertiesAccessor(planner_labels_accessor)
    hierarchy_labels_accessor = HierarchyLabelsAccessor(planner_labels_accessor)
    hierarchy_properties_accessor = HierarchyPropertiesAccessor(planner_labels_accessor)
    return (planner_labels_accessor, node_labels_accessor, node_properties_accessor, hierarchy_labels_accessor, hierarchy_properties_accessor)

def get_calendar_labels_accessor(labels): # TODO: Autogenerated
    calendar_labels_accessor = LabelsAccessor(labels, type='calendar')
    node_labels_accessor = NodeLabelsAccessor(calendar_labels_accessor)
    node_properties_accessor = NodePropertiesAccessor(calendar_labels_accessor)
    hierarchy_labels_accessor = HierarchyLabelsAccessor(calendar_labels_accessor)
    hierarchy_properties_accessor = HierarchyPropertiesAccessor(calendar_labels_accessor)
    return (calendar_labels_accessor, node_labels_accessor, node_properties_accessor, hierarchy_labels_accessor, hierarchy_properties_accessor)

def get_curriculum_labels_accessor(labels): # TODO: Autogenerated
    curriculum_labels_accessor = LabelsAccessor(labels, type='curriculum')
    node_labels_accessor = NodeLabelsAccessor(curriculum_labels_accessor)
    node_properties_accessor = NodePropertiesAccessor(curriculum_labels_accessor)
    hierarchy_labels_accessor = HierarchyLabelsAccessor(curriculum_labels_accessor)
    hierarchy_properties_accessor = HierarchyPropertiesAccessor(curriculum_labels_accessor)
    return (curriculum_labels_accessor, node_labels_accessor, node_properties_accessor, hierarchy_labels_accessor, hierarchy_properties_accessor)
def create_single_sequence_relationship(session, start_node, end_node):
    sequence_rel = neo.create_relationship(session, start_node=start_node, end_node=end_node, label='HAS_NEXT', returns=True)
    return sequence_rel

def sequence_list_of_nodes(session, nodes):
    logging.prod("Creating sequenced relationships between total number of nodes: " + str(len(nodes)))
    sequenced_rels = []
    for i in range(len(nodes)-1):
        sequence_rel = create_single_sequence_relationship(session, nodes[i], nodes[i+1])
        sequenced_rels.append(sequence_rel)
    return sequenced_rels
# We will hard code properties but get labels
def create_now_node(session, state_labels_accessor):
    now_node_dict = {}
    properties = {
        'datetime': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'minute': datetime.now().minute,
        'hour': datetime.now().hour,
        'day': datetime.now().day,
        'month': datetime.now().month,
        'year': datetime.now().year
    }
    now_node = neo.create_node(session, state_labels_accessor.get('now_node'), properties, returns=True) # probably not right
    now_node_dict['node'] = now_node
    now_node_dict['properties'] = properties
    return now_node_dict

def create_current_state(local_calendar_session, local_calendar_dict, local_calendar_labels, local_calendar_properties, local_state_labels, local_state_properties):
    # Create the now node
    now_node_dict = create_now_node(local_calendar_session, local_state_labels)
    local_calendar_dict['now_node'] = now_node_dict
    logging.prod("Created now node")

    # Create the relationship between now and local calendar nodes for the current time_chunk, the current day, the current month, and the current year
    # Get the current time_chunk
    current_time_chunk = local_calendar_dict['time_chunks']['current'] # TODO: This is a placeholder

    return local_calendar_dict
# Functions to prepare properties
def prepare_local_calendar_node(label, start, end, data_dir=None):
    logging.app("Preparing local calendar node properties for type: " + label)
    if label == 'Year':
        logging.pedantic("Preparing local calendar node properties for year: " + str(start))
        properties = {
            'start_date': start.isoformat(),
            'end_date': end.isoformat()
        }
        if data_dir:
            properties['data_dir'] = data_dir
        return properties
    elif label == 'Month':
        logging.pedantic("Preparing local calendar node properties for month: " + str(start.isoformat()))
        properties = {
            'start_month': '{}-{}'.format(start.isoformat().year, start.isoformat().month),
            'end_month': '{}-{}'.format(end.isoformat().year, end.isoformat().month)
        }
        if data_dir:
            properties['data_dir'] = data_dir
        return properties
    elif label == 'Week':
        logging.pedantic("Preparing local calendar node properties for week: " + str(start.isoformat()))
        properties = {
            'start_week': '{}-{}'.format(start.isoformat().year, start.isoformat().isocalendar()[1]),
            'end_week': '{}-{}'.format(end.isoformat().year, end.isoformat().isocalendar()[1])
        }
        if data_dir:
            properties['data_dir'] = data_dir
        return properties
    elif label == 'Date':
        logging.pedantic("Preparing local calendar node properties for date: " + str(start.isoformat()))
        properties = {
            'start_date': start.isoformat(),
            'end_date': end.isoformat()
        }
        if data_dir:
            properties['data_dir'] = data_dir
        return properties
    elif label == 'TimeChunk':
        logging.pedantic("Preparing local calendar node properties for time chunk: " + str(start.isoformat()))
        properties = {
            'start_time_chunk': start.isoformat(),
            'end_time_chunk': end.isoformat()
        }
        if data_dir:
            properties['data_dir'] = data_dir
        return properties
    else:
        return ValueError("Invalid type of local calendar node")

def prepare_local_year_node(year, year_of_calendar, data_dir=None):
    logging.pedantic("Preparing local year node properties for year: " + str(year))
    properties = {
        'year': year,
        'year_of_calendar': year_of_calendar,
    }
    if data_dir:
        properties['data_dir'] = data_dir
    return properties

def prepare_local_month_node(year, month, month_of_calendar, data_dir=None):
    logging.pedantic("Preparing local month node properties for year: " + str(year) + " and month: " + str(month))
    properties = {
        'year_month': '{}-{}'.format(year, month),
        'month_of_calendar': month_of_calendar,
    }
    if data_dir:
        properties['data_dir'] = data_dir
    return properties

def prepare_local_week_node(start_date, week_of_calendar, data_dir=None):
    logging.pedantic("Preparing week node properties for year: " + str(start_date.year) + " and week: " + str(start_date.isocalendar()[1]))
    properties = {
        'year_iso_week': '{}-{}'.format(start_date.year, start_date.isocalendar()[1]),
        'week_of_calendar': week_of_calendar,
    }
    if data_dir:
        properties['data_dir'] = data_dir
    return properties

def prepare_local_date_node(date, d, data_dir=None):
    logging.pedantic("Preparing local date node properties for date: " + date.isoformat())
    properties = {
        'date': date,
        'day_of_calendar': d,
    }
    if data_dir:
        properties['data_dir'] = data_dir
    return properties

def prepare_local_time_chunk_node(time_chunk_start, time_chunk_minutes, time_chunk_for_calendar, data_dir=None):
    logging.pedantic("Preparing local time chunk node properties for start time chunk: " + time_chunk_start.isoformat())
    properties = {
        'time_chunk_start': time_chunk_start,
        'time_chunk_duration_minutes': time_chunk_minutes,
        'time_chunk_of_calendar': time_chunk_for_calendar
    }
    if data_dir:
        properties['data_dir'] = data_dir
    return properties

# Create the constraints
def create_local_calendar_node_constraints(session, labels): # TODO: by file
    local_calendar_node_labels_accessor = get_calendar_labels_accessor(labels)[1]
    local_year_node_label = local_calendar_node_labels_accessor.get('local_year_node_label')
    local_month_node_label = local_calendar_node_labels_accessor.get('local_month_node_label')
    week_node_label = local_calendar_node_labels_accessor.get('week_node_label')
    local_date_node_label = local_calendar_node_labels_accessor.get('local_date_node_label')
    local_time_chunk_node_label = local_calendar_node_labels_accessor.get('local_time_chunk_node_label')
    with session.begin_transaction() as tx:
        local_year_constraint_queries  = [
            f"CREATE CONSTRAINT FOR (y:{local_year_node_label}) REQUIRE y.year IS NOT NULL",
            f"CREATE CONSTRAINT FOR (y:{local_year_node_label}) REQUIRE y.year IS UNIQUE"
        ]
        local_month_constraint_queries = [
            f"CREATE CONSTRAINT FOR (m:{local_month_node_label}) REQUIRE m.year_month IS NOT NULL",
            f"CREATE CONSTRAINT FOR (m:{local_month_node_label}) REQUIRE m.year_month IS UNIQUE"
        ]
        week_constraint_queries = [
            f"CREATE CONSTRAINT FOR (w:{week_node_label}) REQUIRE w.year_iso_week IS NOT NULL",
            f"CREATE CONSTRAINT FOR (w:{week_node_label}) REQUIRE w.year_iso_week IS UNIQUE"
        ]
        local_date_constraint_queries = [
            f"CREATE CONSTRAINT FOR (d:{local_date_node_label}) REQUIRE d.date IS NOT NULL",
            f"CREATE CONSTRAINT FOR (d:{local_date_node_label}) REQUIRE d.date IS UNIQUE"
        ]
        local_time_chunk_constraint_queries = [
            f"CREATE CONSTRAINT FOR (t:{local_time_chunk_node_label}) REQUIRE t.time_chunk_start IS NOT NULL",
            f"CREATE CONSTRAINT FOR (t:{local_time_chunk_node_label}) REQUIRE t.time_chunk_start IS UNIQUE"
        ]
        joined_queries = local_year_constraint_queries + local_month_constraint_queries + week_constraint_queries + local_date_constraint_queries + local_time_chunk_constraint_queries
        for query in  joined_queries:
            logging.query('Running calendar node constraint query: ' + query)
            tx.run(query)
    return True
# Create a local calendar
def create_local_calendar_node(session, labels, label, properties, data_dir=None):
    node_labels_accessor = get_labels_accessor(labels)[1]
    local_calendar_label = node_labels_accessor.get('local_calendar_node_label')
    local_year_label = node_labels_accessor.get('local_year_node_label')
    local_month_label = node_labels_accessor.get('local_month_node_label')
    local_week_label = node_labels_accessor.get('local_week_node_label')
    local_date_label = node_labels_accessor.get('local_date_node_label')
    local_time_chunk_label = node_labels_accessor.get('local_time_chunk_node_label')
    if label == local_calendar_label:
        logging.app(f"Creating local calendar node with label: {label}")
        logging.app(f"Properties: {str(properties)}")
        prepared_properties = prepare_local_calendar_node(properties[0], properties[1], properties[2], properties[3], data_dir)
        node = neo.create_node(session, label, prepared_properties, returns=True)
    elif label == local_year_label:
        prepared_properties = prepare_local_year_node(properties[0], properties[1], properties[2], data_dir)
        node = neo.create_node(session, label, prepared_properties, returns=True)
    elif label == local_month_label:
        prepared_properties = prepare_local_month_node(properties[0], properties[1], properties[2], properties[3], data_dir)
        node = neo.create_node(session, label, prepared_properties, returns=True)
    elif label == local_week_label:
        prepared_properties = prepare_local_week_node(properties[0], properties[1], properties[2], data_dir)
        node = neo.create_node(session, label, prepared_properties, returns=True)
    elif label == local_date_label:
        prepared_properties = prepare_local_date_node(properties[0], properties[1], properties[2], data_dir)
        node = neo.create_node(session, label, prepared_properties, returns=True)
    elif label == local_time_chunk_label:
        prepared_properties = prepare_local_time_chunk_node(properties[0], properties[1], properties[2], properties[3], data_dir)
        node = neo.create_node(session, label, prepared_properties, returns=True)
    else:
        logging.error(f'Cannot create node with label {label} because label not found in local calendar labels.')
        return ValueError("Create node error.")
    return node

# Create relationships
def create_local_calendar_relationship(session, start_node, end_node, label, local_calendar_labels, constraints=True, properties=None):
    try:
        start_label = next(iter(start_node.labels), None)
        end_label = next(iter(end_node.labels), None)
    except:
        logging.error(f"start_node and end_node must be nodes. Got {start_node} and {end_node} instead.")
        return ValueError("Create relationship error.")
    if constraints:
        allowed_calendar_relationship_constraints = create_allowed_calendar_relationship_constraints(local_calendar_labels)
        allowed = any(
            start == start_label
            and end == end_label
            and label in relationships
            for (
                start,
                end,
            ), relationships in allowed_calendar_relationship_constraints.items()
        )
        if allowed:
            logging.info(f"Creating local calendar relationship with constraints: {label} between {start_label} and {end_label}")
            return neo.create_relationship(
                session, start_node, end_node, label, properties, returns=True
            )
        else:
            logging.error(f"Attempted to create disallowed relationship '{label}' between '{start_label}' and '{end_label}'")
            return ValueError("Create relationship error.")
    else:
        logging.warning("Creating local calendar relationship without constraints")
        return neo.create_relationship(
            session, start_node, end_node, label, properties, returns=True
        )

def get_local_calendar_labels(local_calendar_labels):
    node_labels_accessor = get_calendar_labels_accessor(local_calendar_labels)[1]
    hierarchy_labels_accessor = get_calendar_labels_accessor(local_calendar_labels)[3]
    sequence_label_accessor = get_calendar_labels_accessor(local_calendar_labels)[4] # Not implemented yet
    # Node labels
    local_calendar_node_label = node_labels_accessor.get('local_calendar_node_label')
    local_year_node_label = node_labels_accessor.get('local_year_node_label')
    local_month_node_label = node_labels_accessor.get('local_month_node_label')
    local_week_node_label = node_labels_accessor.get('local_week_node_label')
    local_date_node_label = node_labels_accessor.get('local_date_node_label')
    local_time_chunk_node_label = node_labels_accessor.get('local_time_chunk_node_label')
    # Hierarchy labels
    contains_many_hierarchy_label = hierarchy_labels_accessor.get('contains_many_hierarchy_label')
    contains_set_hierarchy_label = hierarchy_labels_accessor.get('contains_set_hierarchy_label')
    contains_single_hierarchy_label = hierarchy_labels_accessor.get('contains_single') # Not implemented yet
    return local_calendar_node_label, local_year_node_label, local_month_node_label, local_week_node_label, local_date_node_label, local_time_chunk_node_label, contains_many_hierarchy_label, contains_set_hierarchy_label, contains_single_hierarchy_label

def get_local_calendar_init_options(init_options):
    local_calendar_options = init_options['local_calendar_options']
    local_calendar_name = local_calendar_options.get('local_calendar_name', 'LocalCalendar')
    create_years = local_calendar_options.get('create_year_nodes', True).lower() in ['true', 't', 'yes', 'y']
    create_months = local_calendar_options.get('create_month_nodes', True).lower() in ['true', 't', 'yes', 'y']
    create_weeks = local_calendar_options.get('create_week_nodes', False).lower() in ['true', 't', 'yes', 'y']
    create_dates = local_calendar_options.get('create_date_nodes', True).lower() in ['true', 't', 'yes', 'y']
    create_time_chunks = local_calendar_options.get('create_time_chunk_nodes', False).lower() in ['true', 't', 'yes', 'y']
    create_data_directories = local_calendar_options.get('create_data_directories', False).lower() in ['true', 't', 'yes', 'y']
    create_sequenced_relationships = local_calendar_options.get('create_sequenced_relationships', False).lower() in ['true', 't', 'yes', 'y']
    sequenced_relationships_options = local_calendar_options.get('sequenced_relationships', {})
    if sequenced_relationships_options:
        sequenced_relationships_options = local_calendar_options.get('sequenced_relationships', {})
        create_sequenced_years = sequenced_relationships_options.get('create_sequenced_years', False).lower() in ['true', 't', 'yes', 'y']
        create_sequenced_months = sequenced_relationships_options.get('create_sequenced_months', False).lower() in ['true', 't', 'yes', 'y']
        create_sequenced_dates = sequenced_relationships_options.get('create_sequenced_dates', False).lower() in ['true', 't', 'yes', 'y']
        create_sequenced_weeks = sequenced_relationships_options.get('create_sequenced_weeks', False).lower() in ['true', 't', 'yes', 'y']
        create_sequenced_time_chunks = sequenced_relationships_options.get('create_sequenced_time_chunks', False).lower() in ['true', 't', 'yes', 'y']
    return local_calendar_name, local_calendar_options, create_years, create_months, create_weeks, create_dates, create_time_chunks, create_data_directories, create_sequenced_relationships, create_sequenced_years, create_sequenced_months, create_sequenced_dates, create_sequenced_weeks, create_sequenced_time_chunks

def create_allowed_calendar_relationship_constraints(local_calendar_labels):
    node_labels_accessor = get_calendar_labels_accessor(local_calendar_labels)[1]
    hierarchy_labels_accessor = get_calendar_labels_accessor(local_calendar_labels)[3]
    local_calendar_node_label = node_labels_accessor.get('local_calendar_node_label')
    local_year_node_label = node_labels_accessor.get('local_year_node_label')
    local_month_node_label = node_labels_accessor.get('local_month_node_label')
    local_week_node_label = node_labels_accessor.get('local_week_node_label')
    local_date_node_label = node_labels_accessor.get('local_date_node_label')
    local_time_chunk_node_label = node_labels_accessor.get('local_time_chunk_node_label')
    contains_many_hierarchy_label = hierarchy_labels_accessor.get('contains_many_hierarchy_label')
    contains_set_hierarchy_label = hierarchy_labels_accessor.get('contains_set_hierarchy_label')
    allowed_calendar_relationship_constraints = {
        (local_calendar_node_label, local_year_node_label): contains_set_hierarchy_label,
        (local_calendar_node_label, local_week_node_label): contains_set_hierarchy_label,
        (local_year_node_label, local_month_node_label): contains_set_hierarchy_label,
        (local_month_node_label, local_date_node_label): contains_set_hierarchy_label,
        (local_week_node_label, local_date_node_label): contains_set_hierarchy_label,
        (local_date_node_label, local_time_chunk_node_label): contains_set_hierarchy_label,
    }
    return allowed_calendar_relationship_constraints
# Function to create a local calendar
def initialise_local_calendar(session, local_calendar_labels, data, init_options, path=None, local_node=None):
    # Options
    local_calendar_name, local_calendar_options, create_years, create_months, create_weeks, create_dates, create_time_chunks, create_data_directories, create_sequenced_relationships, create_sequenced_years, create_sequenced_months, create_sequenced_dates, create_sequenced_weeks, create_sequenced_time_chunks = get_local_calendar_init_options(init_options)
    # Get labels and properties
    local_calendar_node_label, local_year_node_label, local_month_node_label, local_week_node_label, local_date_node_label, local_time_chunk_node_label, contains_many_hierarchy_label, contains_set_hierarchy_label, contains_single_hierarchy_label = get_calendar_labels_accessor(local_calendar_labels)
    # Verify initialisation options
    if create_data_directories and not path:
        logging.error("Path must be provided to create data directories.")
        return ValueError("Initialisation error.")
    # Verify data
    calendar_start_date = data[0]
    calendar_end_date = data[1]
    if calendar_end_date < calendar_start_date:
        logging.error("End date must be after start date")
        return ValueError("Initialisation error.")
    # Create useful variables
    first_year_in_calendar = calendar_start_date.year
    last_year_in_calendar = calendar_end_date.year
    first_month_in_calendar = calendar_start_date.month
    last_month_in_calendar = calendar_end_date.month
    total_years_in_calendar = (calendar_end_date.year - calendar_start_date.year) + 1
    total_months_in_calendar = (calendar_end_date.year - calendar_start_date.year) * 12 + (calendar_end_date.month - calendar_start_date.month) + 1
    total_days_in_calendar = (calendar_end_date - calendar_start_date).days + 1
    if create_time_chunks:
        time_chunk_minutes = int(local_calendar_options.get('time_chunk_minutes', 60))
        time_chunks_in_day = 24*60 / time_chunk_minutes
        normalised_time_chunk_minutes = 24*60 / time_chunks_in_day
        total_time_chunks_in_calendar = total_days_in_calendar * time_chunks_in_day
    if create_data_directories:
        db_path = path
    # Create the local calendar
    local_calendar = {
        'local_calendar_node': None,
        'local_year_nodes': [],
        'local_month_nodes': [],
        'local_date_nodes': [],
        'local_week_nodes': [],
        'local_time_chunk_nodes': [],
        'hierarchy_local_calendar': [], # The highest order nodes in the hierarchy store the relationships between themselves and the next highest order nodes
        'hierarchy_local_year': [],
        'hierarchy_local_month': [],
        'hierarchy_local_week': [],
        'hierarchy_local_date': [],
        'sequenced_local_year_relationships': [],
        'sequenced_local_month_relationships': [],
        'sequenced_local_week_relationships': [],
        'sequenced_local_date_relationships': [],
        'sequenced_local_time_chunk_relationships': [],
    }
    local_calendar_node_properties = [calendar_start_date, calendar_end_date]
    if create_data_directories:
        local_calendar_path = os.path.join(db_path, local_calendar_name)
        os.makedirs(local_calendar_path, exist_ok=True)
        local_calendar_node_properties.append(local_calendar_path)
    local_calendar_node = create_local_calendar_node(session, local_calendar_labels, local_calendar_node_label, local_calendar_node_properties)
    local_calendar['local_calendar_node'] = local_calendar_node
    logging.prod(f"Created local calendar node for {local_calendar_name} with start date {calendar_start_date} and end date {calendar_end_date}")
    # Connect the local calendar to a local node if one is provided
    if local_node: # Not tested yet
        local_calendar_rel = create_local_calendar_relationship(session, local_node, local_calendar_node, contains_single_hierarchy_label, local_calendar_labels)
        logging.prod(f"Connected local calendar to local node: {local_node}")
    # Logic to create years, months, dates, weeks, and time chunks (and periods for planner)
    y = 1
    m = 1
    w = 1
    d = 1
    t = 1
    # Create a year node for every year in the calendar, and a month node for every month in the year, and a date node for every date in the month if initialisation options are set
    for year in range(first_year_in_calendar, last_year_in_calendar + 1):
        if create_years:
            year_properties = [year, y]
            if create_data_directories:
                year_path = os.path.join(local_calendar_path, "cal", str(year))
                logging.app(f"Creating data directory for year {y} at {year_path}")
                os.makedirs(year_path, exist_ok=True)
                year_properties.append(year_path)
            year_node = create_local_calendar_node(session, local_calendar_labels, local_year_node_label, year_properties)
            year_rel = create_local_calendar_relationship(session, local_calendar_node, year_node, contains_many_hierarchy_label, local_calendar_labels)
            local_calendar['local_year_nodes'].append(year_node)
            local_calendar['hierarchy_local_calendar'].append(year_rel)
            logging.prod(f"Created node and relationship for year {y} at {year_path} within local calendar")
        # Create a month node for every month in the calendar
        if year == first_year_in_calendar:
            first_month = first_month_in_calendar
        else:
            first_month = 1
        if year == last_year_in_calendar:
            last_month = last_month_in_calendar
        else:
            last_month = 12
        for month in range(first_month, last_month + 1):
            if create_months:
                month_properties = [year, month, m]
                if create_data_directories:
                    month_path = os.path.join(local_calendar_path, str(year), str(month))
                    logging.app(f"Creating data directory for month {m} at {month_path}")
                    os.makedirs(month_path, exist_ok=True)
                    month_properties.append(month_path)
                month_node = create_local_calendar_node(session, local_calendar_labels, local_month_node_label, month_properties)
                if create_years:
                    month_rel = create_local_calendar_relationship(session, year_node, month_node, contains_set_hierarchy_label, local_calendar_labels)
                else:
                    month_rel = create_local_calendar_relationship(session, local_calendar_node, month_node, contains_many_hierarchy_label, local_calendar_labels)
                local_calendar['local_month_nodes'].append(month_node)
                local_calendar['hierarchy_local_year'].append(month_rel)
                logging.prod(f"Created node and relationship for month {m} at {month_path} within year {y} in local calendar")
            # Create a date node for every date in the month
            if year == first_year_in_calendar and month == first_month_in_calendar:
                first_day = calendar_start_date.day
            else:
                first_day = 1
            if year == last_year_in_calendar and month == last_month_in_calendar:
                last_day = calendar_end_date.day
            else:
                last_day = 31
                if month in [4, 6, 9, 11]:
                    last_day = 30
                if month == 2:
                    if year % 4 == 0:
                        last_day = 29
                    else:
                        last_day = 28
            for day in range(first_day, last_day + 1):
                date = datetime.date(year, month, day)
                if create_dates:
                    date_properties = [date, d]
                    if create_data_directories:
                        date_path = os.path.join(local_calendar_path, str(year), str(month), str(day))
                        logging.app(f"Creating data directory for date {d} at {date_path}")
                        os.makedirs(date_path, exist_ok=True)
                        date_properties.append(date_path)
                    date_node = create_local_calendar_node(session, local_calendar_labels, local_date_node_label, date_properties)
                    if create_months:
                        date_rel = create_local_calendar_relationship(session, month_node, date_node, contains_set_hierarchy_label, local_calendar_labels)
                        local_calendar['local_date_nodes'].append(date_node)
                        local_calendar['hierarchy_local_month'].append(date_rel)
                    else:
                        local_calendar['local_date_nodes'].append(date_node)
                    logging.prod(f"Created node and relationship for date {d} at {date_path} within month {m} in local calendar")
                # Create a time chunk node for every time chunk in the date
                if create_time_chunks:
                    for t_day in range(int(time_chunks_in_day)):
                        time_chunk_start = datetime.datetime.combine(date, time(hour=int(t_day * normalised_time_chunk_minutes / 60), minute=int((t_day * normalised_time_chunk_minutes) % 60)))
                        time_chunk_properties = [time_chunk_start, normalised_time_chunk_minutes, t]
                        if create_data_directories:
                            time_chunk_path = os.path.join(local_calendar_path, str(year), str(month), str(day), str(t_day))
                            logging.app(f"Creating data directory for time chunk {t} at {time_chunk_path}")
                            os.makedirs(time_chunk_path, exist_ok=True)
                            time_chunk_properties.append(time_chunk_path)
                        time_chunk_node = create_local_calendar_node(session, local_calendar_labels, local_time_chunk_node_label, time_chunk_properties)
                        time_chunk_rel = create_local_calendar_relationship(session, date_node, time_chunk_node, contains_set_hierarchy_label, local_calendar_labels)
                        local_calendar['local_time_chunk_nodes'].append(time_chunk_node)
                        local_calendar['hierarchy_local_date'].append(time_chunk_rel)
                        logging.prod(f"Created node and relationship for time chunk {t} at {time_chunk_path} within date {d} in local calendar")
                        t_day += 1
                        t += 1
                d += 1
            m += 1
        y += 1
    # Create week nodes and relationships if create_weeks is set to True
    if create_weeks:
        number_of_days_in_first_week = 7 - calendar_start_date.weekday()
        number_of_days_in_last_week = calendar_end_date.weekday() + 1
        number_of_weeks_in_calendar = (total_days_in_calendar - number_of_days_in_first_week - number_of_days_in_last_week) / 7
        for i in range(int(number_of_weeks_in_calendar) + 2):
            if i == 0:
                week_start_date = calendar_start_date
            elif i == int(number_of_weeks_in_calendar) + 1:
                week_start_date = calendar_end_date - timedelta(days=number_of_days_in_last_week - 1)
            else:
                week_start_date = calendar_start_date + timedelta(days=number_of_days_in_first_week + (i-1)*7)
            week_properties = [week_start_date, w]
            if create_data_directories:
                week_path = os.path.join(local_calendar_path, 'weeks', str(week_start_date.year), str(week_start_date.isocalendar()[1]))
                os.makedirs(week_path, exist_ok=True)
                week_properties.append(week_path)
            week_node = create_local_calendar_node(session, local_calendar_labels, local_week_node_label, week_properties)
            week_rel = create_local_calendar_relationship(session, local_calendar_node, week_node, contains_many_hierarchy_label, local_calendar_labels)
            local_calendar['local_week_nodes'].append(week_node)
            local_calendar['hierarchy_local_calendar'].append(week_rel)
            logging.prod(f"Created week node {w} for week beginning {week_start_date}")
            w += 1
        for date in local_calendar['local_date_nodes']:
            date_iso_week = '{}-{}'.format(date['date'].year, date['date'].isocalendar()[1])
            week_node = [week for week in local_calendar['local_date_nodes'] if week['year_iso_week'] == date_iso_week][0]
            week_rel = create_local_calendar_relationship(session, week_node, date, contains_set_hierarchy_label, local_calendar_labels)
            local_calendar['hierarchy_local_week'].append(week_rel)
            logging.prod(f"Connected date {date['date']} to week {week_node['year_iso_week']}")
    # Create sequenced relationships
    if create_sequenced_relationships:
        if create_sequenced_years:
            sequence_local_year_relationships = sequence_list_of_nodes(session, local_calendar['local_year_nodes'])
            local_calendar['sequenced_local_year_relationships'].append(sequence_local_year_relationships)
            logging.prod("Created sequenced year relationships")
        if create_sequenced_months:
            sequence_local_month_relationships = sequence_list_of_nodes(session, local_calendar['local_month_nodes'])
            local_calendar['sequenced_local_month_relationships'].append(sequence_local_month_relationships)
            logging.prod("Created sequenced month relationships")
        if create_sequenced_weeks:
            sequence_local_week_relationships = sequence_list_of_nodes(session, local_calendar['local_week_nodes'])
            local_calendar['sequenced_local_week_relationships'].append(sequence_local_week_relationships)
            logging.prod("Created sequenced week relationships")
        if create_sequenced_dates:
            sequence_local_date_relationships = sequence_list_of_nodes(session, local_calendar['local_date_nodes'])
            local_calendar['sequenced_local_date_relationships'].append(sequence_local_date_relationships)
            logging.prod("Created sequenced date relationships")
        if create_sequenced_time_chunks:
            sequence_local_time_chunk_relationships = sequence_list_of_nodes(session, local_calendar['local_time_chunk_nodes'])
            local_calendar['sequenced_local_time_chunk_relationships'].append(sequence_local_time_chunk_relationships)
            logging.prod("Created sequenced time chunk relationships")
    # Return the local calendar
    return local_calendar